In [90]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import time
from IPython import display

In [91]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

In [92]:
x_train.shape

(50000, 32, 32, 3)

In [93]:
def discriminators_model():
    model = keras.Sequential()
    model.add(keras.layers.InputLayer((32, 32, 3)))
    model.add(keras.layers.Conv2D(64,kernel_size=3,strides=2,padding="same"))
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Conv2D(128,kernel_size=3,strides=2,padding="same"))
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.MaxPool2D())
    model.add(keras.layers.Dense(1,activation="softmax"))
    return model
discriminators = discriminators_model()

In [94]:
def generator_model():
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=(100,)))
    model.add(keras.layers.Dense(256*4*4))
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.Reshape((4, 4, 256)))
    model.add(keras.layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding="same"))  # 16
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))  # 16
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))  # 32
    model.add(keras.layers.LeakyReLU(0.1))
    model.add(keras.layers.Conv2D(3, (7, 7), padding="same", activation="tanh"))
    return model
generator = generator_model()

In [95]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

In [96]:
def discriminators_loss (real_output,fake_output):
    real = cross_entropy(tf.ones_like(real_output),real_output)
    fake = cross_entropy(tf.zeros_like(fake_output),fake_output)
    return real+fake
def generators_loss (fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

In [99]:
epochs = 200
noise_dim = 100
batch_size = 128

generator_optimazer=keras.optimizers.Adam(1e-4)
discriminators_optimazer=keras.optimizers.Adam(1e-4)

In [100]:
@tf.function
def train_step (images) : 
    noise = tf.random.normal((batch_size,noise_dim))
    with tf.GradientTape() as gan_tape , tf.GradientTape() as dis_tape:
        generate_image = generator(noise)
        real_output = discriminators(images)
        fake_output = discriminators(generate_image)
        gan_loss = generators_loss(fake_output)
        dis_loss = discriminators_loss(real_output,fake_output)
    genertor_gradian = gan_tape.gradient(gan_loss,generator.trainable_variables)
    discriminators_gradian = dis_tape.gradient(dis_loss,discriminators.trainable_variables)
    
    generator_optimazer.apply_gradients(zip(genertor_gradian,generator.trainable_variables))
    discriminators_optimazer.apply_gradients(zip(discriminators_gradian,discriminators.trainable_variables))

In [101]:
def trainig_loop (train_dataset,epochs):
    for epochs in range(epochs):
        start = time.time()
        for batch_image in train_dataset:
            train_step(batch_image)
        display.clear_output(wait=True)
        generate_and_save_images(generator, epoch + 1, seed)
        print(f'Time for epoch {epoch + 1} is {time.time() - start} sec')
    display.clear_output(wait=True)
    generate_and_save_images(generator, epochs, seed)

In [102]:
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.show()

In [103]:
x_train = (x_train - 127.5) / 127.5 

In [104]:
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(60000).batch(batch_size)

trainig_loop(train_dataset.take(), epochs)

NameError: name 'seed' is not defined